In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

from database_operations import DatabaseOperations
from overtake_model import OvertakingModel
from race_data import RaceDataSetup
from race_dataframe import RaceDataframe
from race_sim import RaceSimulator
from evaluation import RaceSimEvaluation, EvaluateMany
from optimisation import Optimisation

import time
from joblib import Parallel, delayed

import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None) 

## Mexico GP

In [2]:
# Initialise all classes
database_ops_mexico = DatabaseOperations(2024, "Mexico City")
race_df_mexico = RaceDataframe(database_ops_mexico)
race_data_mexico = RaceDataSetup(database_ops_mexico, race_df_mexico)
overtake_model_mexico = OvertakingModel(race_df_mexico.race_df)
simulation_mexico = RaceSimulator(race_data_mexico, overtake_model_mexico)

driver = 16 # Charles Leclerc
optimise_mexico = Optimisation(race_data_mexico, overtake_model_mexico, driver)


## Singapore GP

In [3]:
# Initialise all classes
database_ops_singapore = DatabaseOperations(2024, "Marina Bay")
race_df_singapore = RaceDataframe(database_ops_singapore)
race_data_singapore = RaceDataSetup(database_ops_singapore, race_df_singapore)
overtake_model_singapore = OvertakingModel(race_df_singapore.race_df)
simulation_singapore = RaceSimulator(race_data_singapore, overtake_model_singapore)

driver = 16 # Charles Leclerc
optimise_singapore = Optimisation(race_data_singapore, overtake_model_singapore, driver)

In [4]:
def run_singapore_optimisation():
	start_time_bayesian = time.perf_counter()
	strategies_bayesian_singapore = optimise_singapore.bayesian_optimisation(max_iterations=500)
	runtime_bayesian = time.perf_counter() - start_time_bayesian
	
	start_time_genetic = time.perf_counter()
	strategies_genetic_singapore = optimise_singapore.genetic_algorithm_optimisation(population_size=80, generations=10)
	runtime_genetic = time.perf_counter() - start_time_genetic
	

	return strategies_bayesian_singapore, runtime_bayesian, strategies_genetic_singapore, runtime_genetic

## Spanish GP

In [5]:
# Initialise all classes
database_ops_spain = DatabaseOperations(2024, "Barcelona")
race_df_spain = RaceDataframe(database_ops_spain)
race_data_spain = RaceDataSetup(database_ops_spain, race_df_spain)
overtake_model_spain = OvertakingModel(race_df_spain.race_df)
simulation_spain = RaceSimulator(race_data_spain, overtake_model_spain)

driver = 16 # Charles Leclerc
optimise_spain = Optimisation(race_data_spain, overtake_model_spain, driver)

In [6]:
def run_spain_optimisation():
	start_time_bayesian = time.perf_counter()
	strategies_bayesian_spain = optimise_spain.bayesian_optimisation(max_iterations=500)
	runtime_bayesian = time.perf_counter() - start_time_bayesian
	
	start_time_genetic = time.perf_counter()
	strategies_genetic_spain = optimise_spain.genetic_algorithm_optimisation(population_size=80, generations=10)
	runtime_genetic = time.perf_counter() - start_time_genetic
	

	return strategies_bayesian_spain, runtime_bayesian, strategies_genetic_spain, runtime_genetic

## Run Opt

In [7]:
def run_optimisation(optimise, max_iterations, population_size, generations):
	start_time_bayesian = time.perf_counter()
	strategies_bayesian = optimise.bayesian_optimisation(max_iterations=max_iterations)
	runtime_bayesian = time.perf_counter() - start_time_bayesian
	
	start_time_genetic = time.perf_counter()
	strategies_genetic = optimise.genetic_algorithm_optimisation(population_size=population_size, generations=generations)
	runtime_genetic = time.perf_counter() - start_time_genetic
	
	actual_strategy = optimise.get_actual_strategy()

	return strategies_bayesian, runtime_bayesian, strategies_genetic, runtime_genetic, actual_strategy

In [8]:
def print_results(race_name, strategies_bayesian, runtime_bayesian, strategies_genetic, runtime_genetic, actual_strategy_data):
    """
    Prints the results for a given race in a formatted way, including the actual strategy.
    
    Args:
        race_name (str): Name of the race (e.g., "Mexico GP").
        strategies_bayesian: Results from Bayesian optimization (list of dictionaries).
        runtime_bayesian (float): Runtime for Bayesian optimization in seconds.
        strategies_genetic: Results from genetic algorithm optimization (list of dictionaries).
        runtime_genetic (float): Runtime for genetic algorithm optimization in seconds.
        actual_strategy_data (dict): Actual strategy and finishing position of the given driver.
    """
    print(f"=== {race_name} Results ===")
    print(f"Bayesian Optimization Runtime: {runtime_bayesian:.2f} seconds")
    print("Top 10 Bayesian Optimization Strategies:")
    for i, strategy in enumerate(strategies_bayesian[:10], start=1):
        print(f"  {i}. Strategy: {strategy['strategy']}, Final Position: {strategy['position']}")

    print(f"\nGenetic Algorithm Runtime: {runtime_genetic:.2f} seconds")
    print("Top 10 Genetic Algorithm Strategies:")
    for i, strategy in enumerate(strategies_genetic[:10], start=1):
        print(f"  {i}. Strategy: {strategy['strategy']}, Final Position: {strategy['final_position']}")

    print("\nActual Strategy:")
    print(f"  Strategy: {actual_strategy_data['actual_strat']}")
    print(f"  Finishing Position: {actual_strategy_data['actual_finishing_pos']}")

    print("=" * 40)  # Separator

In [11]:
from concurrent.futures import ThreadPoolExecutor

# Run optimizations in parallel using threads
with ThreadPoolExecutor(max_workers=3) as executor:
    future_mexico = executor.submit(run_optimisation, optimise_mexico, 500, 70, 12)
    future_singapore = executor.submit(run_optimisation, optimise_singapore, 500, 70, 12)
    future_spain = executor.submit(run_optimisation, optimise_spain, 500, 70, 12)

    # Retrieve results
    mexico_result = future_mexico.result()
    singapore_result = future_singapore.result()
    spain_result = future_spain.result()

mexico_bayesian, mexico_bayesian_runtime, mexico_genetic, mexico_genetic_runtime, mexico_actual = mexico_result
singapore_bayesian, singapore_bayesian_runtime, singapore_genetic, singapore_genetic_runtime, singapore_actual = singapore_result
spain_bayesian, spain_bayesian_runtime, spain_genetic, spain_genetic_runtime, spain_actual = spain_result

|   iter    |  target   | num_pi... | pit1_lap  | pit1_tyre | pit2_lap  | pit2_tyre | pit3_lap  | pit3_tyre | start_... |
-------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | num_pi... | pit1_lap  | pit1_tyre | pit2_lap  | pit2_tyre | pit3_lap  | pit3_tyre | start_... |
-------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | num_pi... | pit1_lap  | pit1_tyre | pit2_lap  | pit2_tyre | pit3_lap  | pit3_tyre | start_... |
-------------------------------------------------------------------------------------------------------------------------
| 1         | -7.0      | 1.749     | 58.09     | 2.464     | 37.32     | 1.312     | 11.2      | 1.116     | 2.732     |
| 1         | -9.0      | 1.749     | 61.9      | 2.464     | 39.72     | 1.312     | 11.83     | 1.116     | 2.732     |
| 1         | -12.0     

In [12]:
print_results("Mexico GP", mexico_bayesian, mexico_bayesian_runtime, mexico_genetic, mexico_genetic_runtime, mexico_actual)
print_results("Singapore GP", singapore_bayesian, singapore_bayesian_runtime, singapore_genetic, singapore_genetic_runtime, singapore_actual)
print_results("Spain GP", spain_bayesian, spain_bayesian_runtime, spain_genetic, spain_genetic_runtime, spain_actual)

=== Mexico GP Results ===
Bayesian Optimization Runtime: 1967.75 seconds
Top 10 Bayesian Optimization Strategies:
  1. Strategy: {1: 2, 20: 2, 43: 3}, Final Position: 1.0
  2. Strategy: {1: 2, 19: 2, 42: 3}, Final Position: 1.0
  3. Strategy: {1: 2, 20: 2, 41: 3}, Final Position: 1.0
  4. Strategy: {1: 2, 30: 2, 52: 3}, Final Position: 1.0
  5. Strategy: {1: 2, 27: 3, 53: 2}, Final Position: 2.0
  6. Strategy: {1: 2, 29: 2, 55: 3}, Final Position: 2.0
  7. Strategy: {1: 2, 27: 3, 53: 2}, Final Position: 2.0
  8. Strategy: {1: 2, 27: 3, 53: 2}, Final Position: 2.0
  9. Strategy: {1: 2, 31: 2, 51: 3}, Final Position: 2.0
  10. Strategy: {1: 2, 21: 3, 42: 2}, Final Position: 2.0

Genetic Algorithm Runtime: 1971.86 seconds
Top 10 Genetic Algorithm Strategies:
  1. Strategy: {1: 3, 38: 3, 6: 2, 58: 3}, Final Position: 3
  2. Strategy: {1: 3, 50: 3, 26: 3, 12: 2}, Final Position: 3
  3. Strategy: {1: 2, 47: 3, 8: 2, 45: 1}, Final Position: 3
  4. Strategy: {1: 3, 15: 2, 51: 3, 37: 2}, Final 